# Multilingual Analysis Improved

This notebook is for the improvement of multilingual analysis, it is to analyze 15 English translation of 15 Chinese excerpts to set the accuracy baseline for the summaries in the output annotations.

Codes reference: https://colab.research.google.com/drive/1VTi6Z51X6_nAlfSYmIqU7ZnTRVJIjv4j?usp=sharing

## Import the model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir "/content/drive/My Drive/Brahe"
%cd "/content/drive/My Drive/Brahe"

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/Brahe’: File exists
/content/drive/My Drive/Brahe


In [ ]:
!wget https://huggingface.co/Pclanglais/Brahe-AWQ/resolve/main/config.json
!wget https://huggingface.co/Pclanglais/Brahe-AWQ/resolve/main/generation_config.json
!wget https://huggingface.co/Pclanglais/Brahe-AWQ/resolve/main/pytorch_model.bin
!wget https://huggingface.co/Pclanglais/Brahe-AWQ/resolve/main/quant_config.json
!wget https://huggingface.co/Pclanglais/Brahe-AWQ/resolve/main/model.safetensors
!wget https://huggingface.co/Pclanglais/Brahe-AWQ/resolve/main/special_tokens_map.json
!wget https://huggingface.co/Pclanglais/Brahe-AWQ/resolve/main/tokenizer.model
!wget https://huggingface.co/Pclanglais/Brahe-AWQ/resolve/main/tokenizer_config.json

--2024-06-08 00:23:11--  https://huggingface.co/Pclanglais/Brahe-AWQ/resolve/main/config.json
Resolving huggingface.co (huggingface.co)... 13.33.30.76, 13.33.30.49, 13.33.30.114, ...
Connecting to huggingface.co (huggingface.co)|13.33.30.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 801 [text/plain]
Saving to: ‘config.json.12’

config.json.12      100%[===================>]     801  --.-KB/s    in 0s      

2024-06-08 00:23:11 (323 MB/s) - ‘config.json.12’ saved [801/801]

--2024-06-08 00:23:11--  https://huggingface.co/Pclanglais/Brahe-AWQ/resolve/main/generation_config.json
Resolving huggingface.co (huggingface.co)... 13.33.30.76, 13.33.30.49, 13.33.30.114, ...
Connecting to huggingface.co (huggingface.co)|13.33.30.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132 [text/plain]
Saving to: ‘generation_config.json.12’

generation_config.j 100%[===================>]     132  --.-KB/s    in 0s      

2024-06-08 00:23:11 (40.3 M

In [ ]:
%cd ".."

/content/drive/MyDrive


## Install the model

In [ ]:
!pip install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.1/37.1 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
from vllm import LLM, SamplingParams
import os

In [ ]:
def prepare_fabrique_prompt(text):
  prompt = "Text:\n" + str(text) +"\n\n\nAnalysis:\n"
  return prompt

In [ ]:
output_merged_dir="Pclanglais/Brahe-AWQ"
llm = LLM(output_merged_dir, quantization = 'awq')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

WARNING 06-08 00:37:54 config.py:213] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 06-08 00:37:54 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='Pclanglais/Brahe-AWQ', speculative_config=None, tokenizer='Pclanglais/Brahe-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=Pclanglais/Brahe-AWQ)


tokenizer_config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/549 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

INFO 06-08 00:37:59 weight_utils.py:207] Using model weights format ['*.bin']


pytorch_model.bin:   0%|          | 0.00/7.25G [00:00<?, ?B/s]

INFO 06-08 00:43:41 model_runner.py:146] Loading model weights took 6.8084 GB
INFO 06-08 00:43:44 gpu_executor.py:83] # GPU blocks: 993, # CPU blocks: 327
INFO 06-08 00:43:47 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-08 00:43:47 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-08 00:43:56 model_runner.py:924] Graph capturing finished in 10 secs.


In [ ]:
sampling_params = SamplingParams(temperature=0, top_p=1, max_tokens=300, frequency_penalty = 1.2)

## Start analyzing

### Analyze 15 English translation of Chinese excerpts

In [ ]:
text = """

"Akai Shuichi is dead. Killed by Kiel. Let your friend, the former traitor, confirm it. You shouldn't have any problems, right, Scotland?"
"He chose Kiel? I thought it would be you who gave him the 'last kiss', Gin."
The other party made a heavy click of the tongue and hung up the phone. The black-haired, blue-eyed man met the eyes of the eavesdropping stranger next to him, shrugged and smiled, and did not look at the other's embarrassed expression. He turned off the screen, put away his phone, and looked up at the boarding gate in front of him where the queue had already begun.

"... Passengers departing from Kuala Lumpur International Airport to Tokyo, Japan, please note that your flight JL414 is now boarding. Please take your personal belongings, show your boarding pass, and board at Gate 3. Have a nice trip. Thank you!"
--
<Bourbon, investigate the truth behind Akai Shuichi's death. Time is money!>
<Received. >
"You really can't keep me quiet, Rum."
In a villa in the suburbs of London, a blond, dark-skinned man put down his phone, leaned back, and stretched. He stretched his neck and fingers slightly, casually opened a page on the laptop next to him, and booked a flight from Heathrow Airport to Haneda Airport in Japan in seven days. First class.
He plugged a USB drive into another computer, dragged the file out of it with the mouse, and pulled it into a software to start parsing. The white light and black words on the screen were reflected in his drooping purple-gray eyes. The man muttered inaudibly in a room where no one was except himself.
"After I get rid of you, I will have to find someone who will not let people rest in peace even if he dies. You better be worth the money."

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.98s/it, Generation Speed: 23.87 toks/s]

['Summary: The text is about a conversation between two characters, Scotland '
 'and Gin, discussing the death of Akai Shuichi.\n'
 'Enunciation: Dialogue\n'
 'Tone: Serious\n'
 'Genre: Mystery/Detective Fiction\n'
 'Intertextuality: Conversation between two people discussing a crime '
 'investigation\n'
 'Speech standard: Standard language with some informal expressions\n'
 'Literary form: Dialogue/conversation']


In [ ]:
text = """

"Please sit down." The young man smiled with a mouthful of white teeth, and his cheerful look really looked like those businessmen who were calculating. "My name is Huang Zihongfan, you can just call me Huang Zi."

Cao Enqi took off the hat that covered his face, put it on the table casually, and bowed slightly, "Cao Enqi, please give me more advice."

"You're welcome." Huang Zihongfan's expression moved slightly, motioned him to sit down, patted his shoulder carelessly, and poured a cup of hot tea, "This is a small shop. I hope you can understand that the hospitality is not good."

"Shopkeeper Huang is polite." Cao Enqi did not refuse, drank it in one gulp, and added with a slightly surprised and appreciative look, "Just now, the servant said that the shopkeeper would not add more seats, and after listening to it, Your invitation is no doubt. I think there will be no other person in the shopkeeper of the Caiyun Tower. In low voice, "Huang Zi Hongfan, an old man asked me to send a letter to Caiyun Tower." "The old man?" He asked calmly, and continued to clear the tea for himself, and took a sip of the porcelain cup. Shaking. He dangled his head and put his face with a smile on his face. "The news of Apu was too late. Nanbomen was destroyed." As soon as this remark, Cao Enqi was cold.

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.98s/it, Generation Speed: 23.64 toks/s]

['Summary: Cao Enqi meets Huang Zihongfan in a small shop, where they discuss '
 'the destruction of Nanbomen.\n'
 'Enunciation: Dialogue\n'
 'Tone: Serious\n'
 'Genre: Historical fiction\n'
 'Intertextuality: News report\n'
 'Speech standard: Standard language with some informal expressions\n'
 'Literary form: Conversation/dialogue\n'
 'Active character: Cao Enqi, Huang Zihongfan']


In [ ]:
text = """

However, this Friday morning was destined to be a day out of the norm. The woman stretched, revealing the pleasing curves of her female body, and the beautiful scenery at the moment, the man enjoyed these two things. Suddenly, the woman's expression changed, and the next second her teeth began to tremble. Others could hardly notice such details, but the man next to her could. He could see from her eyes that the woman's eyes were completely occupied by fear.

The woman said tremblingly in a voice that the man had never heard before: "Johnny, look, is that a person lying over there?"

Marty went past the reporter who was sitting on the lawn in a daze, raised the yellow cordon, and took a look at the victim as usual. Except for some special cases, such as the naked woman wearing deer antlers while kneeling, most of the victims were relatively ordinary and inconspicuous. Just like this victim, who was about the same age as him, the injury was on the back of the head, so when he was lying on the front, there was almost no scar, but his eyes were wide open, as if he was still shocked that death had come like this, and death solidified the shock of that moment, and finally formed into a piece of paper in the camera of the on-site investigators.

"When was the time of death?"

"Based on the liver temperature, the time of death was about ten o'clock last night."

Marty wrote something in his notebook, which was neither big nor small. After writing, he subconsciously looked in the direction he came from.

"What's the cause of death?"

"Preliminary observation is that the middle meningeal artery was damaged due to skull fracture, which in turn caused epidural hematoma." The old policeman has seen at least eighty such causes of death caused by blunt objects hitting the back of the head. Despite this, Marty still wrote it down. After writing, he looked in the same direction again, then he squatted down and carefully observed the only wound. It didn't look big. It was probably because the angle was very tricky and hit the artery.

As if he had a sense of it, he turned around and Rust was walking towards him. He walked very straight, and Marty's heart skipped a beat. In a trance, he thought it was still 1995. Fortunately, he reacted in time and reached out to lift the warning belt for Rust. Rust cooperated well with him and got in nimbly with a slight lowering of his body.

Marty dutifully reported what he had just recorded in his ear. Rust stood beside the body and stared at it. Marty stared at his profile and brow bone, looking thoughtful, so he believed in his observation: "The wallet is gone. Maybe it was just a robbery. The robber didn't hit hard, but just hit the artery."

"Isn't this place suitable for close people to come? Such a beautiful scenery." Rust said. In the background, Johnny was brushing his girlfriend's hair aside and consoling her attentively, firmly telling her that everything would be fine.

An ordinary policeman in uniform walked towards them: "Although there is no wallet, we found some receipts in the deceased's pocket. I think you should take a look at this one."

It was a receipt for a meal at the cafeteria of Princeton Teaching Hospital. The time was yesterday, and two hamburgers and fries were ordered. Generally speaking, the hospital implements a meal delivery system for patients, which means that it is very likely that the deceased was a hospital employee.

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.17s/it, Generation Speed: 21.86 toks/s]

['Summary: The text describes a man observing a woman who is fearful and then '
 'turns his attention to a victim of death.\n'
 'Narrative arc: Suspense\n'
 'Enunciation: Third-person narrative\n'
 'Tone: Serious, investigative\n'
 'Genre: Crime fiction\n'
 'Intertextuality: News report, police investigation report\n'
 'Speech standard: Standard, formal language used by the characters in '
 'conversation with each other']


In [ ]:
text = """

Dante took longer than expected to get the water. He walked in with the water slowly, his eyes slightly probing, scanning Tracy's eyes and lower abdomen. Tracy felt uncomfortable being stared at by him, frowning in confusion, with an ominous premonition rising in her heart. At the same time, she also prepared herself mentally to receive Dante's confession that he had messed up some small things again.

"What's wrong?"

"Drink water first." Dante stuffed the water cup into her hand to stop her from talking. Only after watching her drink it with his own eyes, taking the water cup from her hand and putting it away, did he continue: "Do you feel dizzy and weak?"

"Yeah." Seeing his serious attitude, Tracy also took it seriously.

"Fever, nausea, vomiting, sleepiness and dreaminess, do you have these?"

"All of them..."

"Have you had a lot of mood swings recently?"

"What do you want to say, Dr. Dante?" Tracy couldn't help but tease.

Dante's face looked bad, as if he was considering more appropriate words. After hesitating for a long time, he gave up resistance and uttered the answer word by word.

"...Are you pregnant?"

"...Pregnant? How is that possible?" Tracy's smile froze on her face. She denied this absurd guess without thinking, but her hand covered her lower abdomen as if to cover up her guilty conscience. The skin covered by her palm was hot from the inside out. The abdominal cavity turned into a warm fish tank, nourishing a goldfish that was rushing around. It was hitting and jumping again and again, and it was unclear whether it was from the twitching of stomach spasms or the fetal movement of an unknown life.

"I'm just guessing. Go to bed quickly, maybe you'll be fine after a good night's sleep." Dante prevaricated with a lame excuse, urging Tracy to lie down and close her eyes quickly, and even carefully tucked Tracy's outstretched hand into the quilt.

What a fool. Tracy sighed softly. Of course, she failed to drive away the annoying thoughts and force herself to fall asleep.

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.81s/it, Generation Speed: 23.11 toks/s]

["Summary: Tracy is surprised by Dante's question about her being pregnant, "
 "but denies it. She can't fall asleep because of her thoughts.\n"
 'Narrative arc: Suspense\n'
 'Enunciation: Dialogue\n'
 'Tone: Serious\n'
 'Genre: Drama\n'
 'Speech standard: Standard language with some slang words\n'
 'Literary form: Conversation/dialogue']


In [ ]:
text = """

The summer in Sanbrek is mild and rainless, but Dion, sitting in the carriage, seems a little restless. He adjusts his sitting position again and again, sometimes leaning on the window and staring at the White Dragon City in the distance, and sometimes leaning on the back of the chair and staring at the roof in a daze, his hands constantly stroking his soft velvet seat covers, commanding them to change color between lake blue and dark blue.
"You just lost your temper, my prince."
Dion stretched out his hand and waved, saying, "No need to say it, and it's not the first time in front of you."
"It's my honor to have this privilege, sir." Terrence blinked, drew a small circle in the air with his hand, and saluted Dion.
"You are really annoying sometimes, like now."
He leaned forward slightly, took off his gloves and clasped his hands, and passed his ten fingers through his fingers. Terrence obediently cooperated with him, letting the calluses on their hands kiss each other. The carriage was bumpy, and Dion left his seat and sat on Terrence. The loyal captain of the guards pressed Dion's back with his unoccupied left hand so that his captain could sit more steadily. Terrence heard Dion sigh with satisfaction, and buried his head in Terrence's neck. The golden hair brushed his nose, and the brown-haired knight smelled the light.
"I love you."
"Why do you say this suddenly?"
"Because I love you, just like you love me."
"Then can my beloved Dion explain what happened just now?"
"You will know, but before that, you have to think about what happened when we were in the monastery school."
"Why?"
"The dragon knights of Sanbrek need to fight a prepared battle."

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.18s/it, Generation Speed: 23.29 toks/s]

['Summary: Dion is restless in the carriage and talks to Terrence about their '
 'relationship.\n'
 'Enunciation: Dialogue\n'
 'Tone: Affectionate, playful\n'
 'Genre: Romance\n'
 'Speech standard: Informal, conversational\n'
 'Literary form: Conversation/dialogue\n'
 'Active character: Dion, Terrence']


In [ ]:
text = """

"Good morning, Master Soga, Madam Soga." The sheen of the marble wall made her feel a little chilly in the sultry summer. Master Soga and Madam Soga were reading newspapers. "Isa, you're here?" Madam Soga put down the newspaper, "Sit down quickly, eat first if you're hungry."
Isa forced a smile: "Thank you, Madam, I'm not hungry yet."
Madam Soga held up the newspaper, "Isa, you're already eighteen years old, it's too late if you don't get married. Women's youth depreciates quickly." Isa suppressed her nausea and made a shy expression: "Husband... Madam is right." Seeing her reaction, the master smiled complacently: "Women are troublesome, don't end up being unable to have children and unable to get married." When the master used a knife and fork, the fat on his face always trembled with the movement of his arms. He treated food and money like people of lower status than himself, and couldn't wait to enjoy them.
"The master is right." Isa felt that he couldn't eat anything anymore.
"In that case," the lady turned a little towards Isa happily after hearing this, "There's a gentleman who's looking for a wife recently, Isa, do you want to go?"
Isa opened her eyes wide: "Now?"
"That gentleman is very powerful, you'll be blessed if you marry him." Mrs. Soga loved to pretend to be jealous of Isa when she handed the mess to her. She spent hundreds of thousands of dollars to maintain her face, but the muscles on her apple cheeks and lips were so stiff that it was hard to tell if she was really smiling.
Isa also smiled: "Thank you so much, master and lady. You're so kind to me and willing to marry me to a good family. I really don't know how to repay you."
"Isa, I'm so happy you said that," the lady tilted her eyebrows and squeezed out a strong smile at her, "Our family is financially tight recently, but we will do our best for your dowry. After all, it's our Isa's wedding, so you have to look like a noble lady, right?"
Yes, like a noble lady. Her parents' inheritance had long been divided up by relatives, and her value in the Soga family was about the same as a painting. How much value would she have after getting married?
"I will spend my whole life repaying your kindness." Yisha said lies and wiped away her sincere tears.

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.59s/it, Generation Speed: 22.57 toks/s]

['Summary: Isa is being pressured by her adoptive parents to get married, but '
 'she feels uneasy about the prospect.\n'
 'Narrative arc: Pressure and unease\n'
 'Enunciation: Dialogue\n'
 'Tone: Serious, tense\n'
 'Genre: Realistic fiction\n'
 'Speech standard: Standard, polite conversation\n'
 'Literary form: Conversation/dialogue']


In [ ]:
text = """

The expedition team went deep into Jingyu and lost contact. After two weeks of being out of contact, only one male survivor was rescued in a village near the mouth of Yulao River, but he was unconscious. After the survivor regained consciousness, he told a rather bizarre story. He claimed that the expedition team had discovered the ruins of the Shang Dynasty in the depths of the Qinling Mountains. Later, under the leadership of this person, the team went there again, but only a broken bronze engraving was found. The traces were blurred and it was made about three thousand years ago, but whether it was an old object of the Shang Dynasty remains to be studied. "

Strange, although the things of the Shang Dynasty are precious, they are not impossible to explore. Moreover, I have never seen the patterns on these bronze plates. It sounds a bit conceited, but I am a graduate student in the Department of Archaeology after all, and my main direction happens to be medieval civilization. So I have not seen it, which means that this matter is not taken seriously, so that no one continues to study it.

My curiosity has expanded unprecedentedly. I opened the remaining three pieces of letter paper. The ink is very beautiful, but what really surprised me is the content on it.

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.06s/it, Generation Speed: 24.66 toks/s]

['\n'
 'Summary: The expedition team went deep into Jingyu and lost contact. After '
 'two weeks, only one survivor was rescued and he told a bizarre story about '
 'discovering the ruins of the Shang Dynasty. The narrator is curious about '
 'the content of the remaining letters.\n'
 'Trope: Mysterious ruins, lost civilization\n'
 'Narrative arc: Suspense\n'
 'Enunciation: First-person narrative\n'
 'Tone']


In [ ]:
text = """

After his mother died, Cao Rui became a ghost in the East Palace. No one could mess with him, after all, he was the emperor's son, but everyone knew about Madam Zhen's affairs. His Majesty was not the kind of strict person who would not vent his anger on others. It was better to say that the Cao family was naturally a bit promiscuous, pursuing a kind of exhaustive love and hate, like the paint applied when the Luoyang Palace was newly built. It was difficult to wash off once it was stained, and there was always a smell after washing, so people had to avoid it for the rest of their lives. The East Palace was thus deserted, but it was not entirely a bad thing - Cao Rui was troubled by a strange disease at that time and could not speak a whole long sentence. This is how they discovered this: One morning, about a month after Madam Zhen's death, the maids helped him wash and comb his hair. The comb was old and his hair was too long and inconvenient, and unfortunately one of the teeth was broken that day. Cao Rui closed his eyes and heard the crisp sound very close to his ear, and felt strongly that it was unlucky, just like a string breaking. He opened his eyes and told them to change it quickly: "Why is it broken? Go and change it--change it..." When he said this, the words that followed seemed to be stuck and he couldn't utter them. Cao Rui covered his mouth in astonishment. The women who had been holding his hair looked at him in fear and trepidation, and after realizing that something was wrong, they all knelt down. More combs fell to the ground in panic. They thought that they had angered the Marquis of Pingyuan, and Cao Rui sat on the couch and thought: Why are you so nervous? But he couldn't say this sentence, and could only look at them. The girls lowered their heads, and the tassels decorating their hair buns on their heads trembled in the windless palace, making a lingering sound, until he couldn't bear it anymore and stood up and walked around the people in front of him and walked outside, his dark hair and the hem of his clothes dragging on the ground. He walked out the door and called the eunuchs in the palace. He couldn't see anyone today. Today... today... Cao Rui opened his mouth, decided not to appear to be a stutterer, and divided the sentence into three parts. "Uncomfortable." He pointed at himself, "Let them," he pointed outside again, "go back." The next day everyone in the palace knew about it. Mrs. Guo sent someone to express her condolences. Cao Rui was very respectful to the female official. He knew that his nominal mother was actually coming on behalf of his biological father, so he just wanted to send her away quickly. He explained to her that he was fine, and his words were accidentally broken into several pieces. The female official understood and left. A few days later, Mrs. Guo sent more things, which were all sealed and stored.

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.38s/it, Generation Speed: 22.83 toks/s]

["Summary: After his mother's death, Cao Rui becomes a ghost in the East "
 'Palace and is known for his promiscuity. One day, he discovers that he can '
 'no longer speak in full sentences.\n'
 'Trope: Promiscuity\n'
 'Narrative arc: Character development\n'
 'Enunciation: Third-person narrative\n'
 'Tone: Tragic\n'
 'Genre: Historical novel\n'
 'Intertextuality: Historical biography\n'
 'Speech standard']


In [ ]:
text = """

Just one step away.

Ma Hanqing stared at the gilded brocade box on the table, and picked up the sharp arrow that was shooting straight at his chest. The fine sweat on his forehead proved that he had gone through a lot of hardships to get here, leaving the palace soldiers behind. Although the study was unguarded, it was full of traps, and he could die if he was not careful. But the target was in front of him. As long as he took one step closer, he would chop the harmful wood and destroy the forged secret letter. Wu Taishi would not be able to produce the "evidence" of General Ma's treason, and the 100,000 Ma family soldiers would be cleared of all charges, and General Ma would no longer have to suffer imprisonment.

A sudden pain came from his heart.

Damn it. Ma Hanqing's heart sank. Those people must have caught the illusion he had released, and they would be here soon. He had not yet found the flaw in the trap array. Instead of waiting to die, it would be better to fight to the death. Compared with General Ma's innocence and the future of the 100,000 Ma family soldiers, the death of Ma Hanqing was nothing.

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.06s/it, Generation Speed: 24.62 toks/s]

['Summary: Ma Hanqing is in a dangerous situation, but he is determined to '
 'find a way out.\n'
 'Trope: Heroic sacrifice\n'
 'Narrative arc: Suspense\n'
 'Enunciation: Third-person narrative\n'
 'Tone: Determined, desperate\n'
 'Genre: Historical fiction\n'
 'Intertextuality: Action movie scene\n'
 'Speech standard: Standard literary language with some poetic elements (e.g. '
 '"fine sweat" and "']


In [ ]:
text = """

When the journey was almost halfway through, the two finally saw the first inhabited village along the way.

However, the place that should have been a place for travelers to replenish their luggage was actually in a very bad situation. Alechino asked Columbia to stay on the saddle pad, and he got off the horse, walked to the left front, and walked slowly forward holding the reins. Ragged people wandered around the town, like empty shells, and they hardly even moved their eyes. Only when they saw the visitors dressed in bright clothes, those "people" who were as thin as skeletons would open their mouths slightly and show a little bit of human vitality.

"Why did they become like this?" Columbia asked while looking at the crowd of beggars gradually surrounding them on horseback.

"War. The Queen is marching westward. The food, money, and manpower needed for the war will eventually become a heavy burden for the workers. It brings prestige to the royal family and benefits to the nobles, and then the taxes and famine hidden in the shadows are pressed on every bottom-level people." Alechino responded to the girl's words, took out a few silver coins from the inside of the fur robe, and handed them to the beggar in front of him.

"Why don't they go somewhere else?"

"They have nowhere to go. Since birth, serfs have been bound to the land. They belong to the farmland, the manor, and the lord-but not to themselves."

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.56s/it, Generation Speed: 24.16 toks/s]

['Summary: The protagonist and his companion encounter a village in a poor '
 'state due to war.\n'
 'Narrative arc: Reflective\n'
 'Enunciation: Dialogue\n'
 'Tone: Serious, somber\n'
 'Genre: Historical fiction\n'
 'Intertextuality: Historical documentary or political commentary\n'
 'Speech standard: Standard, formal language\n'
 'Literary form: Conversation/dialogue between characters']


In [ ]:
text = """

At the beginning of the new calendar, Sanbrek and the two neighboring countries revised a new law. The three-nation alliance that reunited in the war is now working hard to restore the broken world. Even a hundred years later, various old calendar legends are circulating on the road. Many things are forgotten facts, and many things are clues that conceal the truth. A young apprentice from the monastery bid farewell to his hometown and went from the remote edge of the Dragon's Nest to the Holy White Dragon City in New Oleflem, the new capital of Sanbrek. Like an ascetic monk, he walked north along the gravel road in simple and sloppy cloth shoes.

The chocobo was led behind him, his only travel companion, and he couldn't bear to let this poor little guy carrying heavy objects add to his weight. So he slowly walked out of the plain and crossed the waterfall until they saw the watchtower on the cliff by the sea.

He opened the book in his bag, which recorded the appearance of the world map before the war. He was in this barren land with sparse crops. A hundred years ago, there were still mutated monsters making trouble, and two hundred years ago, it was even submerged in the mysterious high concentration of ether and difficult to move forward. He repeatedly chewed the word "ether" in his mouth, full of curiosity and fear about this unknown thing from the past.

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.07s/it, Generation Speed: 24.58 toks/s]

['Summary: A young apprentice from a monastery sets out on a journey to the '
 'new capital of Sanbrek, reflecting on the past and contemplating the '
 'mysterious substance known as ether.\n'
 'Trope: Journey of self-discovery\n'
 'Narrative arc: Reflective and introspective\n'
 'Enunciation: Third-person narrative\n'
 'Tone: Reflective, curious\n'
 'Genre: Fantasy fiction\n'
 'Intertextual']


In [ ]:
text = """

"Old friend, ten years of imprisonment are over. How do you feel? Reborn? Or a little reluctant?"
The voice was frivolous but powerful, not like that of the mortal world. It was sent into the listener's skull, with an irresistible sense of aggression, rolling in the inner cavity of the ear, causing the whole person to resonate with it.
"Hermes." Odysseus recognized the person and looked at him with some surprise, "Why are you here?"
"I came to see our poor puppy, is it living well on this desolate island."
Hermes' tone was slightly raised, as careless as he was. He was happily spinning in the air on his flying shoes, like a full-winged bird. But Odysseus knew that he was different from Athena, not similar to an owl, but more casual and free. Hermes was willing to pay for one day at a time and live for one moment at a time. He was more willing to be clever than to be upright and wise. He looked like an approachable god, but in fact he was difficult to deal with on another level. However, no matter how idle he is, he would not tease him for no reason. There must be a reason, a sign, or a guide, Odysseus thought. As smart as Odysseus was, he quickly understood the intention of the angel of the gods. "You persuaded Calypso to let me go, so is this the will of the gods? In this case, can I guess that in the near future, I will finally be able to return to Ithaca as I wish?"

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.72s/it, Generation Speed: 23.92 toks/s]

['Summary: Odysseus is surprised to see Hermes on the island and asks if it '
 'means he will be able to return home soon.\n'
 'Enunciation: Dialogue\n'
 'Tone: Casual, frivolous\n'
 'Genre: Epic poetry\n'
 'Speech standard: Informal, colloquial\n'
 'Literary form: Conversation/dialogue\n'
 'Active character: Odysseus, Hermes']


In [ ]:
text = """

They had been sitting in the monitoring room for an hour and a half. Alana and Chilton stared at the monitoring screen from beginning to end. Chilton had a recorder in his hand. Margo, who said she was not interested, came into the room at six o'clock with some food and a drink for Alana. Jack left half an hour ago. He didn't have time to watch a mental patient drawing, a silent child scribbling, and a poor guy who had the misfortune of connecting them through the womb staring at the floor.

"There are always more important things in the world than observing autistic children - and adults," Chilton declared in surrender. "See you later, everyone."

He also hurried away, leaving Alana and Margo staring at each other.

"So this is autism." Margo turned to the screen and saw Will standing up and collecting his daughter's paintbrushes. "No wonder she didn't say anything."

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.95s/it, Generation Speed: 25.34 toks/s]

['Summary: Alana and Chilton have been observing autistic children for an hour '
 'and a half, while Margo brings food to Alana. Jack leaves, and Chilton also '
 'leaves. Margo comments on the silence of the children.\n'
 'Enunciation: Dialog\n'
 'Tone: Serious\n'
 'Genre: Psychological drama\n'
 'Intertextuality: Scientific article\n'
 'Speech standard: Standard/neutral\n'
 'Literary form: Conversation/dialogue\n']


In [ ]:
text = """

When the young demon hunter found Virgil, he was curled up in the corner of the first floor of the old Sparta house. The interior of the old house, which had not been fully renovated, was empty. The young Virgil was wearing a dirty curtain and curled up in the shadow under the stairs. Nero scratched his head. He squatted down to barely see the whole picture in this small shadow. The young man muttered that he must seal this place with cement to prevent mice from making nests here, while trying to pick Virgil up carefully.

The child slept very lightly, and he woke up suddenly almost as soon as Nero's hand touched him.

"Hey, hey, hey, I mean no harm, okay?" Nero raised his hands, Virgil pulled out his sword with difficulty, and the tip of the Yama Sword pointed at Nero's nose tremblingly: "You don't remember me? God, what is going on - do you remember how you got here?"

"Shut up!" The young Virgil scolded him harshly: "Tell me your purpose!"

"My purpose is to take you back... No, not that kind, fuck, my purpose is to take you to a safe place..." Nero sighed: "It sounds more suspicious... Do you want to see Dante? Although he is already too old."

Virgil stared at the man in front of him, he knew that something was wrong with him, crossing, or time going backwards. The child woke up on the outskirts of Red Tomb City, which was very different from what he remembered. When he trudged over, everything he saw had told him the answer. There were not so many ruined buildings in Red Tomb City, and the decorations and advertisements on the street were different from what he had seen.

"Who are you?" the boy asked: "What is the relationship between me and you?"

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.19s/it, Generation Speed: 23.86 toks/s]

['Summary: The young demon hunter finds Virgil in an old house and tries to '
 'take him to a safe place.\n'
 'Narrative arc: Suspense\n'
 'Enunciation: Dialogue between two characters\n'
 'Tone: Suspenseful\n'
 'Genre: Fantasy/Demonic fiction\n'
 'Speech standard: Informal, colloquial language\n'
 'Literary form: Conversation/dialogue between characters\n'
 'Active character: Nero, Virgil']


In [ ]:
text = """

The morning mist gradually dissipated, and Astoria could see tents everywhere. It seemed that the World Cup hosted by the UK was very popular. Although the Ministry of Magic was "governed by some stupid people", Astoria still admired that it could focus on so many people from all over the world and ensure their safety.

Around them, there were several large tents with American flags, and the door was full of wine bottles, which made Daphne raise her upper lip; there was also a white building with a cylindrical pointed roof (others called it a "dome"), and several big men were holding something like a whip outside, swinging it on the grill, and the meat made a sizzling sound when it was cooked. Astoria couldn't help but open her mouth.

They continued to move forward and found more tents from different countries on the road. A young man who looked like a Frenchman smiled and said something foreign to Astoria, but was driven away by Daphne. Astoria was attracted by a kind of unknown card played by several witches from Sierra Leone. She was about to bend down to see what was going on, and Daphne patted her shoulder.

  """

final_prompt = prepare_fabrique_prompt(text)

prompts = [final_prompt]

outputs = llm.generate(prompts, sampling_params)

list_generated = []
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    list_generated.append(generated_text)

from pprint import pprint

pprint(list_generated)

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.04s/it, Generation Speed: 24.78 toks/s]

['Summary: Astoria and Daphne are exploring the tents at a World Cup event '
 'hosted by the UK.\n'
 'Enunciation: Third-person narrative\n'
 'Tone: Curious, observant\n'
 'Genre: Travelogue\n'
 'Intertextuality: Travel guide, tourist brochure\n'
 'Speech standard: Standard, polite conversation\n'
 "Literary form: Description of a place and characters' actions/thoughts\n"
 'Active character: Astoria,']
